<a href="https://colab.research.google.com/github/ardominguezm/emoji-hill-cipher-paper-/blob/main/examples.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Emoji–Hill Cipher Examples
This notebook reproduces the encryption and decryption examples from the paper.

In [1]:
import numpy as np
import emoji
import pandas as pd

# Hybrid alphabet (N=40) consistent with the paper
alphabet = [
    "A","B","C","D","E","F","G","H","I","J",
    "K","L","M","N","O","P","Q","R","S","T",
    "U","V","W","X","Y","Z"," ", "!", "0","1",
    "2","3","4",
    emoji.emojize(":brain:"),
    emoji.emojize(":rocket:"),
    emoji.emojize(":bar_chart:"),
    emoji.emojize(":robot:"),
    emoji.emojize(":locked:"),
    emoji.emojize(":smiling_face_with_smiling_eyes:"),
    emoji.emojize(":puzzle_piece:")             # PAD
]

N = len(alphabet)
phi = {ch: i for i, ch in enumerate(alphabet)}
phi_inv = {i: ch for i, ch in enumerate(alphabet)}

print("Alphabet size:", N)
print("Sample indices:", {ch: phi[ch] for ch in [emoji.emojize(':brain:'), 'A', 'X']})
print("Emojis-Hill alphabet:", {ch: phi[ch] for ch in alphabet})

Alphabet size: 40
Sample indices: {'🧠': 33, 'A': 0, 'X': 23}
Emojis-Hill alphabet: {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25, ' ': 26, '!': 27, '0': 28, '1': 29, '2': 30, '3': 31, '4': 32, '🧠': 33, '🚀': 34, '📊': 35, '🤖': 36, '🔒': 37, '😊': 38, '🧩': 39}


In [2]:
# Key matrix (same as in the paper)
K = np.array([[1, 2, 3],
              [0, 1, 4],
              [5, 6, 0]])
det = int(round(np.linalg.det(K)))
print("det(K) =", det)

# Inverse over Z_40
K_inv = np.linalg.inv(K).astype(float)
K_adj = np.round(det * K_inv).astype(int)  # adjugate
print("K^-1 adj:\n", K_adj) # since det=1
K_inv_mod = (K_adj % N)  # since det=1
print("K^-1 mod 40:\n", K_inv_mod)

det(K) = 1
K^-1 adj:
 [[-24  18   5]
 [ 20 -15  -4]
 [ -5   4   1]]
K^-1 mod 40:
 [[16 18  5]
 [20 25 36]
 [35  4  1]]


In [7]:
def encode_message(msg):
    return [phi[ch] for ch in msg]

def decode_message(indices):
    return ''.join(phi_inv[i] for i in indices)

def blockify(seq, r=3):
    # Padding with PAD if needed
    while len(seq) % r != 0:
        seq.append(phi[emoji.emojize(":puzzle_piece:")])
    return [seq[i:i+r] for i in range(0, len(seq), r)]

def encrypt_block(block):
    x = np.array(block)
    y = K.dot(x) % N
    return y.tolist()

def decrypt_block(block):
    y = np.array(block)
    x = K_inv_mod.dot(y) % N
    return x.astype(int).tolist()

#Example 1:Encryption

In [8]:
# Example 1: Encryption
plaintext = [emoji.emojize(":brain:")] + list("DATA SCIENCE IS FUN!") \
             + [emoji.emojize(":rocket:"), emoji.emojize(":bar_chart:")]

# Encode and blockify
x_indices = encode_message(plaintext)
blocks = blockify(x_indices, r=3)

rows = []
for j, b in enumerate(blocks, 1):
    y_raw = K.dot(b)
    y_mod = (y_raw % N).tolist()
    decoded = [phi_inv[i] for i in y_mod]
    rows.append([j, b, y_raw.tolist(), y_mod, decoded])

import pandas as pd
df_enc = pd.DataFrame(rows, columns=["j","x (indices)","Kx (pre-mod)","Kx mod 40","Decoded"])
df_enc

,j,x (indices),Kx (pre-mod),Kx mod 40,Decoded
0,1,"[33, 3, 0]","[39, 3, 183]","[39, 3, 23]","[🧩, D, X]"
1,2,"[19, 0, 26]","[97, 104, 95]","[17, 24, 15]","[R, Y, P]"
2,3,"[18, 2, 8]","[46, 34, 102]","[6, 34, 22]","[G, 🚀, W]"
3,4,"[4, 13, 2]","[36, 21, 98]","[36, 21, 18]","[🤖, V, S]"
4,5,"[4, 26, 8]","[80, 58, 176]","[0, 18, 16]","[A, S, Q]"
5,6,"[18, 26, 5]","[85, 46, 246]","[5, 6, 6]","[F, G, G]"
6,7,"[20, 13, 27]","[127, 121, 178]","[7, 1, 18]","[H, B, S]"
7,8,"[34, 35, 39]","[221, 191, 380]","[21, 31, 20]","[V, 3, U]"


In [9]:
# Ciphertext final- Example 1
ciphertext = ''.join(''.join(row[-1]) for row in rows)
print("Ciphertext:\n", ciphertext)

Ciphertext:
 🧩DXRYPG🚀W🤖VSASQFGGHBSV3U


# Example 2: Decryption


In [10]:

import numpy as np
import pandas as pd
import emoji

# --- auxiliar functions ---
def encode_message(msg):
    return [phi[ch] for ch in msg]

def decode_message(indices):
    return [phi_inv[i] for i in indices]

def blockify(seq, r=3):
    return [np.array(seq[i:i+r]) for i in range(0, len(seq), r)]

# --- Alfabeth (N=40) ---
alphabet = [
    "A","B","C","D","E","F","G","H","I","J",
    "K","L","M","N","O","P","Q","R","S","T",
    "U","V","W","X","Y","Z"," ", "!", "0","1",
    "2","3","4",
    emoji.emojize(":brain:"),
    emoji.emojize(":rocket:"),
    emoji.emojize(":bar_chart:"),
    emoji.emojize(":robot:"),
    emoji.emojize(":locked:"),
    emoji.emojize(":smiling_face_with_smiling_eyes:"),
    emoji.emojize(":puzzle_piece:") # PAD
]

N = len(alphabet)
phi = {ch: i for i, ch in enumerate(alphabet)}
phi_inv = {i: ch for i, ch in enumerate(alphabet)}

# --- # Key matrix and inverse (same as in the paper) ---
K = np.array([[1,2,3],[0,1,4],[5,6,0]])
K_inv = np.array([[-24,18,5],[20,-15,-4],[-5,4,1]])  # inverse
K_inv_mod = K_inv % N                                # inverse modulo N

# --- Ciphertext in blocks ---
cipher_blocks = [
    ["U","M","M"],                               # 1
    ["A", emoji.emojize(":puzzle_piece:"),"M"],  # 2
    ["T","0","J"],                               # 3
    ["T","E",emoji.emojize(":puzzle_piece:")],   # 4
    ["!","F","D"],                               # 5
    ["X","2","V"],                               # 6
    ["P","E",emoji.emojize(":smiling_face_with_smiling_eyes:")], # 7
    ["Y","G","3"],                               # 8
    ["D","H",emoji.emojize(":rocket:")],         # 9
    [emoji.emojize(":smiling_face_with_smiling_eyes:"), emoji.emojize(":locked:"),"0"], # 10
    ["C","C","0"],                               # 11
    ["Y","M","4"],                               # 12
    ["A","U","N"],                               # 13
    ["S","L","4"],                               # 14
    ["S","S","N"]                                # 15
]

# flatten
cipher_example = [ch for block in cipher_blocks for ch in block]

# Encode & blockify
cipher_indices = encode_message(cipher_example)
blocks_c = blockify(cipher_indices, r=3)

# --- Decrypt block by block ---
rows_dec = []
for j, b in enumerate(blocks_c, 1):
    y = b
    x_raw = K_inv.dot(y)          # producto antes del mod
    x_mod = (x_raw % N).tolist()  # N modulo reduction
    decoded = decode_message(x_mod)
    rows_dec.append([j, y.tolist(), x_raw.tolist(), x_mod, decoded])

df_dec = pd.DataFrame(rows_dec, columns=["j","y (indices)","K^-1 y (pre-mod)","K^-1 y mod 40","Decoded"])

# Mostrar resultados
import IPython.display as disp
disp.display(df_dec)


,j,y (indices),K^-1 y (pre-mod),K^-1 y mod 40,Decoded
0,1,"[20, 12, 12]","[-204, 172, -40]","[36, 12, 0]","[🤖, M, A]"
1,2,"[0, 39, 12]","[762, -633, 168]","[2, 7, 8]","[C, H, I]"
2,3,"[19, 28, 9]","[93, -76, 26]","[13, 4, 26]","[N, E, ]"
3,4,"[19, 4, 39]","[-189, 164, -40]","[11, 4, 0]","[L, E, A]"
4,5,"[27, 5, 3]","[-543, 453, -112]","[17, 13, 8]","[R, N, I]"
5,6,"[23, 30, 21]","[93, -74, 26]","[13, 6, 26]","[N, G, ]"
6,7,"[15, 4, 38]","[-98, 88, -21]","[22, 8, 19]","[W, I, T]"
7,8,"[24, 6, 31]","[-313, 266, -65]","[7, 26, 15]","[H, , P]"
8,9,"[3, 7, 34]","[224, -181, 47]","[24, 19, 7]","[Y, T, H]"
9,10,"[38, 37, 28]","[-106, 93, -14]","[14, 13, 26]","[O, N, ]"


In [12]:
#Plaintext final-Example 2
plaintext_recovered = ''.join(''.join(row[-1]) for row in rows_dec)
print(plaintext_recovered)

🤖MACHINE LEARNING WITH PYTHON IS AMAZING! 🔒😊📊
